In [1]:
import cv2
import numpy as np
import math 

In [ ]:
import cv2
import numpy as np

#loading img
image = cv2.imread("cropped_1.jpg")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#inverting so shapes are white
_, threshold_img = cv2.threshold(gray_image, 220, 255, cv2.THRESH_BINARY_INV)

#finding contrs chain_approch-simple only keeps the end points of lines,shapes
contrs, _ = cv2.findContours(threshold_img , cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# Contour 0:
# [[[200 200]]
#  [[200  50]]
#  [[ 50  50]]
#  [[ 50 200]]]

# Contour 1:
# [[[300 300]]
#  [[350 350]]
#  [[300 400]]]

# Contour 2:
# [[[450 100]]
#  [[500 150]]
#  [[450 200]]]

for i,contr in enumerate(contrs) :  #((0,contrs[0]) , (1,contrs[1]))
    if i == 0 :                     #skip the whole frame contour
        continue
    if cv2.contourArea(contr) < 100: #skip tiny noise contours
        continue

    #approximating the shape - ignore imperfections
    eps = 0.02 * cv2.arcLength(contr , True)
    approx = cv2.approxPolyDP(contr,eps , True)

    #drawing approximated contour
    cv2.drawContours(image , [approx] , 0 , (0,0,0) ,4)

    x,y,w,h = cv2.boundingRect(approx)
    #getting the center
    M = cv2.moments(contr)
    if M["m00"] != 0 :
        x_mid = int(M["m10"] / M["m00"])
        y_mid = int(M["m01"] / M["m00"]) #int conversion to get correct pixles 
    else :
        x_mid = x + w // 2
        y_mid = y + h // 2

    #Shape classification
    if len(approx) == 3  :
        shape = "Triangle"

    elif len(approx) == 4  :
        ar = w / h
        shape = "Square" if 0.9 <= ar <= 1.1 else "Rectangle"

    elif len(approx) == 5  :
        shape = 'Pentagon'

    elif len(approx) == 6  :
        shape = 'Hexagon'
    
    else :
        print(len(approx))
        shape = "Circle"

    #------
    #Color 
    #------
    #Masking
    mask = np.zeros(image.shape[:2], dtype="uint8") #same shape neglecting the color 
    cv2.drawContours(mask, [contr], -1, 255, -1) #draw contours with same shape 

    mean_color = cv2.mean(image , mask=mask)[:3] #BGR avg
    b,g,r = mean_color

    # Simple rule-based color naming
    if r > 150 and g < 100 and b < 100:
        color_name = "Red"
    elif g > 150 and r < 100 and b < 100:
        color_name = "Green"
    elif b > 150 and r < 100 and g < 100:
        color_name = "Blue"
    elif r > 150 and g > 150 and b < 100:
        color_name = "Yellow"
    elif r > 150 and g > 150 and b > 150:
        color_name = "White"
    elif r < 80 and g < 80 and b < 80:
        color_name = "Black"
    else:
        color_name = "Other"
    
    #labeling
    label = f"{color_name} {shape}"
    (text_w, text_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_DUPLEX, 0.7, 2)

    text_x = x_mid - text_w // 2
    text_y = y_mid

    box_coords = (
        (text_x - 5, text_y - text_h - 5),
        (text_x + text_w + 5, text_y + 5)
    )

    cv2.rectangle(image, box_coords[0], box_coords[1], (255, 255, 255), cv2.FILLED)
    cv2.putText(image, label, (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 100, 100), 2)

cv2.imwrite("labeled_shapes_1.jpg", image)
cv2.imshow("Shapes" , image)
cv2.waitKey(0)
cv2.destroyAllWindows()
